In [8]:
@file:DependsOn("ai.timefold.solver:timefold-solver-core:1.14.0")
@file:DependsOn("com.fasterxml.jackson.module:jackson-module-kotlin:2.17.1")
@file:DependsOn("org.jetbrains.kotlinx:kotlinx-coroutines-core:1.7.3")

import ai.timefold.solver.core.api.score.buildin.hardsoft.HardSoftScore
import ai.timefold.solver.core.api.score.stream.Constraint
import ai.timefold.solver.core.api.score.stream.ConstraintFactory
import ai.timefold.solver.core.api.score.stream.ConstraintProvider
import ai.timefold.solver.core.api.score.stream.Joiners
import ai.timefold.solver.core.api.score.stream.ConstraintCollectors
import ai.timefold.solver.core.api.domain.solution.PlanningSolution
import ai.timefold.solver.core.api.domain.solution.PlanningScore
import ai.timefold.solver.core.api.domain.entity.PlanningEntity
import ai.timefold.solver.core.api.domain.lookup.PlanningId
import ai.timefold.solver.core.api.domain.variable.PlanningVariable
import ai.timefold.solver.core.api.domain.solution.ProblemFactCollectionProperty
import ai.timefold.solver.core.api.domain.solution.PlanningEntityCollectionProperty
import ai.timefold.solver.core.api.domain.valuerange.ValueRangeProvider
import ai.timefold.solver.core.api.solver.Solver
import ai.timefold.solver.core.api.solver.SolverFactory
import ai.timefold.solver.core.config.solver.SolverConfig
import java.time.LocalTime
import java.time.DayOfWeek
import java.time.Duration

In [9]:
data class Location(
    val name: String,
    val latitude: Double,
    val longitude: Double
)

data class Skill(
    val name: String
)

data class Vehicle(
    val id: Long,
    val licensePlate: String,
    val capacity: Int = 1
)

data class Agent(
    @PlanningId
    val id: Long,
    val name: String,
    val skills: List<Skill>,
    val homeLocation: Location,
    val vehicle: Vehicle,
    val weeklyWorkloadHours: Int = 40
)

@PlanningEntity
data class Meeting(
    @PlanningId
    val id: Long,
    val client: String,
    val location: Location,
    val requiredSkills: List<Skill>,
    val durationHours: Int,
    val startTime: LocalTime,
    val endTime: LocalTime,
    @PlanningVariable(valueRangeProviderRefs = ["agentRange"])
    var assignedAgent: Agent? = null
) {
    constructor() : this(
        0L, "", Location("", 0.0, 0.0), emptyList(), 1,
        LocalTime.MIDNIGHT, LocalTime.MIDNIGHT
    )
}

@PlanningSolution
data class Schedule(
    @ProblemFactCollectionProperty
    @ValueRangeProvider(id = "agentRange")
    val agents: List<Agent>,

    @PlanningEntityCollectionProperty
    val meetings: List<Meeting>
) {
    @PlanningScore
    var score: HardSoftScore? = null

    constructor() : this(emptyList(), emptyList())
}

In [10]:
val numberOfAgents = 5
val numberOfSkills = 3
val numberOfMeetings = 20
val locationNamePrefix = "Location"

val predefinedSkills = listOf(
    Skill("Fire Certification"),
    Skill("Bodyguard Certification"),
    Skill("First Aid Certification")
)

fun generateVehicles(count: Int): List<Vehicle> {
    return (1..count).map {
        Vehicle(it.toLong(), "XYZ-00$it")
    }
}

fun generateAgents(
    count: Int,
    skills: List<Skill>,
    homeLocation: Location
): List<Agent> {
    val vehicles = generateVehicles(count)
    return (1..count).map { id ->
        Agent(
            id = id.toLong(),
            name = "Agent_$id",
            skills = skills.shuffled().take((1..skills.size).random()),
            homeLocation = homeLocation,
            vehicle = vehicles[id - 1]
        )
    }
}

fun generateLocations(count: Int): List<Location> {
    val genevaLatitude = 46.2044
    val genevaLongitude = 6.1432
    return (1..count).map { id ->
        Location(
            name = "$locationNamePrefix$id",
            latitude = genevaLatitude + (0.01 * (Math.random() - 0.5)),
            longitude = genevaLongitude + (0.01 * (Math.random() - 0.5))
        )
    }
}

fun generateMeetings(
    count: Int,
    skills: List<Skill>,
    locations: List<Location>
): List<Meeting> {
    return (1..count).map { id ->
        val requiredSkillCount = (1..skills.size).random()
        val requiredSkills = skills.shuffled().take(requiredSkillCount)
        val location = locations.random()
        val duration = (1..3).random()
        val startHour = (8..20).random()
        val startTime = LocalTime.of(startHour, 0)
        val endTime = startTime.plusHours(duration.toLong())
        Meeting(
            id = id.toLong(),
            client = "Client_$id",
            location = location,
            requiredSkills = requiredSkills,
            durationHours = duration,
            startTime = startTime,
            endTime = endTime
        )
    }
}

val homeLocation = Location("Home_Geneva", 46.2044, 6.1432)
val agents = generateAgents(numberOfAgents, predefinedSkills, homeLocation)
val meetingLocations = generateLocations(10)
val meetings = generateMeetings(numberOfMeetings, predefinedSkills, meetingLocations)

In [11]:
class SecurityConstraintProvider : ConstraintProvider {
    override fun defineConstraints(constraintFactory: ConstraintFactory): Array<Constraint> {
        return arrayOf(
            noOverlap(constraintFactory),
            skillRequirements(constraintFactory),
            workloadLimit(constraintFactory),
            minimizeTotalTravelTime(constraintFactory),
            preferAgentRoomStability(constraintFactory)
        )
    }

    private fun noOverlap(constraintFactory: ConstraintFactory): Constraint {
        return constraintFactory.forEachUniquePair(Meeting::class.java,
            Joiners.equal(Meeting::assignedAgent),
            Joiners.overlapping(
                { it.startTime.toSecondOfDay() },
                { it.endTime.toSecondOfDay() }
            )
        )
            .penalize(HardSoftScore.ONE_HARD)
            .asConstraint("No overlapping meetings")
    }

    private fun skillRequirements(constraintFactory: ConstraintFactory): Constraint {
        return constraintFactory.forEach(Meeting::class.java)
            .filter { meeting ->
                meeting.assignedAgent != null &&
                        !meeting.requiredSkills.all { skill -> meeting.assignedAgent!!.skills.contains(skill) }
            }
            .penalize(HardSoftScore.ONE_HARD)
            .asConstraint("Agent lacks required skills")
    }

    private fun workloadLimit(constraintFactory: ConstraintFactory): Constraint {
        return constraintFactory.forEach(Meeting::class.java)
            .groupBy(Meeting::assignedAgent, ConstraintCollectors.sum(Meeting::durationHours))
            .filter { agent, totalHours -> agent != null && totalHours > 40 }
            .penalize(HardSoftScore.ONE_HARD, { _, totalHours -> totalHours - 40 })
            .asConstraint("Weekly workload limit")
    }

    private fun minimizeTotalTravelTime(constraintFactory: ConstraintFactory): Constraint {
        return constraintFactory.forEach(Meeting::class.java)
            .filter { it.assignedAgent != null }
            .penalize(HardSoftScore.ONE_SOFT,
                { meeting ->
                    val agentHome = meeting.assignedAgent!!.homeLocation
                    val distance = euclideanDistance(agentHome, meeting.location)
                    (distance * 10).toInt()
                }
            )
            .asConstraint("Minimize travel time")
    }

    private fun preferAgentRoomStability(constraintFactory: ConstraintFactory): Constraint {
        return constraintFactory.forEach(Meeting::class.java)
            .filter { it.assignedAgent != null }
            .reward(HardSoftScore.ONE_SOFT,
                { meeting ->
                    val agentHome = meeting.assignedAgent!!.homeLocation
                    val distance = euclideanDistance(agentHome, meeting.location)
                    (100 - distance).toInt()
                }
            )
            .asConstraint("Prefer agent room stability")
    }

    private fun euclideanDistance(loc1: Location, loc2: Location): Double {
        val dx = loc1.latitude - loc2.latitude
        val dy = loc1.longitude - loc2.longitude
        return Math.sqrt(dx * dx + dy * dy)
    }
}

In [5]:
val solverConfig = SolverConfig()
    .withSolutionClass(Schedule::class.java)
    .withEntityClasses(Meeting::class.java)
    .withConstraintProviderClass(SecurityConstraintProvider::class.java)
    .withTerminationSpentLimit(Duration.ofSeconds(30))

val solverFactory: SolverFactory<Schedule> = SolverFactory.create(solverConfig)

val problem = Schedule(agents, meetings)

println("Solving the problem...")
val solver: Solver<Schedule> = solverFactory.buildSolver()
val solution: Schedule = solver.solve(problem)
println("Solving finished with score: ${solution.score}")

Solving the problem...
Solving finished with score: -10hard/1980soft


In [6]:
fun formatTime(time: LocalTime): String {
    return time.format(java.time.format.DateTimeFormatter.ofPattern("HH:mm"))
}

val assignedMeetings = solution.meetings.filter { it.assignedAgent != null }
val unassignedMeetings = solution.meetings.filter { it.assignedAgent == null }

println("Assigned Meetings:")
assignedMeetings.groupBy { it.assignedAgent }.forEach { (agent, meetings) ->
    println("${agent?.name}:")
    meetings.sortedBy { it.startTime }.forEach { meeting ->
        println("  - ${meeting.client} (${formatTime(meeting.startTime)} - ${formatTime(meeting.endTime)}) - Skills: ${meeting.requiredSkills.joinToString(", ") { it.name }}")
    }
    println()
}

println("Unassigned Meetings:")
unassignedMeetings.forEach { meeting ->
    println("- ${meeting.client} (${formatTime(meeting.startTime)} - ${formatTime(meeting.endTime)}) - Skills: ${meeting.requiredSkills.joinToString(", ") { it.name }}")
}

Assigned Meetings:
Agent_3:
  - Client_13 (09:00 - 12:00) - Skills: Fire Certification, First Aid Certification
  - Client_8 (13:00 - 14:00) - Skills: Bodyguard Certification, Fire Certification, First Aid Certification
  - Client_9 (15:00 - 17:00) - Skills: First Aid Certification, Bodyguard Certification, Fire Certification
  - Client_16 (18:00 - 20:00) - Skills: First Aid Certification, Fire Certification
  - Client_3 (19:00 - 20:00) - Skills: Fire Certification, Bodyguard Certification
  - Client_1 (20:00 - 21:00) - Skills: Bodyguard Certification, Fire Certification
  - Client_5 (20:00 - 21:00) - Skills: Fire Certification

Agent_4:
  - Client_2 (08:00 - 10:00) - Skills: First Aid Certification
  - Client_19 (14:00 - 16:00) - Skills: Fire Certification, Bodyguard Certification
  - Client_20 (16:00 - 19:00) - Skills: Fire Certification
  - Client_11 (19:00 - 21:00) - Skills: Fire Certification, Bodyguard Certification, First Aid Certification

Agent_2:
  - Client_4 (08:00 - 11:00) 

In [7]:
import ai.timefold.solver.core.api.score.buildin.hardsoft.HardSoftScore
import ai.timefold.solver.core.api.solver.SolutionManager

val solutionManager = SolutionManager.create(solverFactory)
val scoreExplanation = solutionManager.explain(solution)

println("Score: ${solution.score}")

println("\nDetailed Score Explanation:")
println("Hard Score: ${(solution.score as HardSoftScore).hardScore()}")
println("Soft Score: ${(solution.score as HardSoftScore).softScore()}")

println("\nConstraint Matches:")
scoreExplanation.constraintMatchTotalMap.forEach { (constraintId, total) ->
    println("${constraintId}: ${total.score}")
}

println("\nIndictments:")
scoreExplanation.indictmentMap.forEach { (indictedObject, indictment) ->
    println("$indictedObject: ${indictment.score}")
    indictment.constraintMatchSet.forEach { match ->
        println("  - ${match.constraintId}: ${match.score}")
    }
}

Score: -10hard/1980soft

Detailed Score Explanation:
Hard Score: -10
Soft Score: 1980

Constraint Matches:
unnamed.package/Agent lacks required skills: -8hard/0soft
unnamed.package/Minimize travel time: 0hard/0soft
unnamed.package/No overlapping meetings: -2hard/0soft
unnamed.package/Prefer agent room stability: 0hard/1980soft
unnamed.package/Weekly workload limit: 0hard/0soft

Indictments:
Meeting(id=1, client=Client_1, location=Location(name=Location5, latitude=46.20714240341011, longitude=6.142335018358459), requiredSkills=[Skill(name=Bodyguard Certification), Skill(name=Fire Certification)], durationHours=1, startTime=20:00, endTime=21:00, assignedAgent=Agent(id=3, name=Agent_3, skills=[Skill(name=Fire Certification), Skill(name=First Aid Certification), Skill(name=Bodyguard Certification)], homeLocation=Location(name=Home_Geneva, latitude=46.2044, longitude=6.1432), vehicle=Vehicle(id=3, licensePlate=XYZ-003, capacity=1), weeklyWorkloadHours=40)): -1hard/99soft
  - unnamed.package